In [1]:
%reload_ext typedb_jupyter

In [2]:
%typedb help

Available commands: connect, database, transaction, help
TODO: Print subcommand help


In [3]:
%typedb connect open core 127.0.0.1:1729 admin password

Opened connection to: 127.0.0.1:1729


In [4]:
%typedb database create typedb_jupyter_sample

Created database  typedb_jupyter_sample


In [5]:
%typedb database list

Databases:  typedb_jupyter_sample, moo, test_jupyter, jupyter-test, typedb-iam


In [6]:
%typedb database delete typedb_jupyter_sample

Deleted database  typedb_jupyter_sample


In [7]:
%typedb database list

Databases:  moo, test_jupyter, jupyter-test, typedb-iam


In [8]:
%typedb database recreate typedb_jupyter_sample

Recreated database  typedb_jupyter_sample


In [9]:
%typedb database list

Databases:  typedb_jupyter_sample, moo, test_jupyter, jupyter-test, typedb-iam


In [10]:
# This is not implemented yet: %typedb database schema test_jupyter

In [11]:
%typedb database recreate typedb_jupyter_sample

Recreated database  typedb_jupyter_sample


In [12]:
%typedb transaction open typedb_jupyter_sample schema

Opened schema transaction on database 'typedb_jupyter_sample'  


In [13]:
%%typeql 
define
    attribute name, value string;
    entity person, owns name;


SyntaxError: expected '(' (display.py, line 47)

In [ ]:
%typedb transaction commit

In [ ]:
%typedb transaction open typedb_jupyter_sample write

In [ ]:
%%typeql
insert 
$p isa person, has name "James";

In [ ]:
%typedb transaction commit

In [ ]:
%typedb transaction open typedb_jupyter_sample read 

In [ ]:
%%typeql 
match $owner isa! $owner_type;

In [ ]:
print(_typeql_result)

In [ ]:
%%typeql 
match $owner isa! $owner_type; entity $owner_type;
fetch {
    "attributes": { $owner.* }
};

In [ ]:
%typedb transaction close